#### Check to see if the connection to the timescale database is working


In [1]:
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine, text

load_dotenv()
PASSWORD = os.getenv("PGPASSWORD")
USER = os.getenv("PGUSER")
DB = os.getenv("PGDATABASE")
PORT = os.getenv("PGPORT")
HOST = os.getenv("PGHOST")
MODE = os.getenv("PGSSLMODE")

In [ ]:
CONNECTION = f"timescaledb://{USER}:{PASSWORD}@{HOST}:{PORT}/{DB}?sslmode={MODE}"

In [ ]:
engine = create_engine(CONNECTION)
conn = engine.connect()
cursor = conn.execute(text("select extname, extversion from pg_extension;"))
for extension in cursor:
    print(extension)
conn.close()

('plpgsql', '1.0')
('pg_stat_statements', '1.10')
('timescaledb', '2.17.2')
('timescaledb_toolkit', '1.18.0')
('postgres_fdw', '1.1')
('pg_buffercache', '1.4')


#### Making a connection to the database and creating a table for the weather data


In [ ]:
engine = create_engine(CONNECTION)
conn = engine.connect()

cursor = conn.execute(text("select extname, extversion from pg_extension;"))
for extension in cursor:
    print(extension)

# Drop the existing weather table
drop_table_sql = "DROP TABLE IF EXISTS weather;"
conn.execute(text(drop_table_sql))
print("Existing weather table dropped.")

# Recreate the weather table
create_table_sql = """
CREATE TABLE IF NOT EXISTS weather (
    time        TIMESTAMPTZ NOT NULL,        -- Timestamp of the weather data
    city_name   TEXT,                        -- Name of the city
    condition   TEXT,                        -- Weather condition (e.g., "Clear", "Cloudy")
    temperature FLOAT,                       -- Temperature (°C or °F based on units)
    humidity    INTEGER,                     -- Humidity (percentage)
    pressure    FLOAT,                       -- Atmospheric pressure (hPa)
    rain        FLOAT,                       -- Rain volume (mm)
    snow        FLOAT,                       -- Snow volume (mm)
    wind_speed  FLOAT,                       -- Wind speed (m/s or miles/h)
    wind_deg    FLOAT,                       -- Wind direction (degrees)
    cloudiness  INTEGER,                     -- Cloudiness percentage
    sunrise     BIGINT,                      -- Sunrise time in nanoseconds since Unix epoch
    sunset      BIGINT,                      -- Sunset time in nanoseconds since Unix epoch
    PRIMARY KEY (time, city_name)            -- Unique key based on city and time
);
"""

# Execute the query to create the table
conn.execute(text(create_table_sql))
print("Weather table created successfully!")

# Convert to a hypertable
create_hypertable_sql = """
SELECT create_hypertable('weather', 'time');
"""
conn.execute(text(create_hypertable_sql))
print("Weather table converted to hypertable successfully!")

#### Checking total number of rows in the table


In [ ]:
# Create SQLAlchemy engine and connect
engine = create_engine(CONNECTION)
conn = engine.connect()

count_rows_sql = "SELECT COUNT(*) FROM weather;"
result = conn.execute(text(count_rows_sql)).fetchone()
print(f"Total number of rows in the weather table: {result[0]}")

# Close the connection
conn.close()

Total number of rows in the weather table: 111160
